In [1]:
require '../lib/pca'
require 'nyaplot'
require 'nmatrix/rowcol' # enable arbitrary row selection
require 'csv'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

false

In [2]:
csv = CSV.read "data/wine.data"
IRuby.table csv[0..4]

1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065
1,13.2,1.78,2.14,11.2,100,2.65,2.76,.26,1.28,4.38,1.05,3.4,1050
1,13.16,2.36,2.67,18.6,101,2.8,3.24,.3,2.81,5.68,1.03,3.17,1185
1,14.37,1.95,2.5,16.8,113,3.85,3.49,.24,2.18,7.8,.86,3.45,1480
1,13.24,2.59,2.87,21,118,2.8,2.69,.39,1.82,4.32,1.04,2.93,735


In [3]:
data = csv.map {|row| row[1..13].map(&:to_f) }.to_nm
labels = csv.map {|row| row[0].to_i }.to_nm

categories = labels.to_set.to_a

counts = categories.map do |category|
  data.row(labels =~ category).rows
end

IRuby.table category: categories, samples: counts

category,samples
1,59
2,71
3,48


In [4]:
pca = PCA.new components: 2, scale_data: true
data_pca = pca.fit_transform data

IRuby.table data_pca.row(0..4).to_a

-3.3074209742892213,-1.439402253182295
-2.203249813420225,0.3324550711941798
-2.509660694761864,-1.0282507242941947
-3.7464971904980016,-2.7486183908585367
-1.0060704895977053,-0.8673840350754064


In [5]:
plot = Nyaplot::Plot.new
plot.x_label('PC1')
plot.y_label('PC2')
plot.legend(true)

colors = Nyaplot::Colors.qual

categories.each_with_index do |category, i|
  samples = data_pca.row labels =~ category
  sc = plot.add(:scatter, samples.col(0).to_a, samples.col(1).to_a)
  sc.color(colors[i])
  sc.title(category)
end

plot

#<Nyaplot::Plot:0x007fe6f3c5fc88 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fe6f3bad510 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"rgb(102,194,165)", :title=>1}, :data=>"e70a0f49-0bf4-446d-9eba-29c7bbe3f00e"}, @xrange=[[-4.300652282433131], [-0.4609089685901877]], @yrange=[[-2.7486183908585367], [0.7672676364432762]]>, #<Nyaplot::Diagram:0x007fe6f3b1dd98 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"rgb(252,141,98)", :title=>2}, :data=>"bad8f3ef-4916-4248-8bcc-722958993ebe"}, @xrange=[[-2.5550197688136387], [2.531835081161829]], @yrange=[[-0.2594666264180434], [3.8608927330254605]]>, #<Nyaplot::Diagram:0x007fe6f6126e90 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"rgb(141,160,203)", :title=>3}, :data=>"4726a17e-dfd1-4480-b6a4-106d5df4f4a2"}, @xrange=[[1.0452334187292003], [4.268597576898801]], @yrange=[[-3.5052019369175067], [0.4583418817494623]]>], :options=>{:x_label=>"PC1", :y_label=>"PC2", :legend=>true, :zoom=>true, :width=>800, :xrange=>[[-4.300652282433131], [4.268597576898801]], :yrange=>[[-3.5052019369175067], [3.8608927330254605]]}}>

In [6]:
pca.explained_variance_ratio.map {|v| v.round(4)}

#<NMatrix:0x007fe6f39b6658 shape:[1,2] dtype:object stype:dense>